In [4]:
from pulp import *

proces_time = {'A1':{'TTC':6,'TTL':2,'C':6,'P': 4,'MM':5},
               'A2':{'TTC':10,'TTL':8,'C':4,'P':2,'MM':3},
               'A3':{'TTC':4,'TTL':3,'C':5,'P':7,'MM':4}}

# 
M = len(proces_time.keys()) # M = 3
N = len(proces_time['A1'].keys()) # N = 5

machine_names = [key for key in proces_time.keys()] # ['A1', 'A2', 'A3']
job_names = [key for key in proces_time['A1'].keys()] # ['TTC', 'TTL', 'C', 'P', 'MM']

# Ranges where indexes are defined
jobs = range(1,N+1)
machines = range(1,M+1)

# convert data to dictionary {(i,k): value}   (i=job, k=machine)
p = {(i,k): proces_time[machine_names[k-1]][job_names[i-1]] for i in jobs for k in machines}

# Create the model
prob = LpProblem("flowshop", LpMinimize)

# Variables
x = LpVariable.dicts("x", ((i,k) for i in range(N+1) for k in range(N+1)), cat='Binary')
s = LpVariable.dicts("s", ((k,j) for k in range(N+1) for j in range(M+1)), lowBound=0)

# Constraints
# One job is scheduled only once
for k in jobs: 
    prob += lpSum(x[(i,k)] for i in jobs) == 1

# Each possible order is used only once
for i in jobs: 
    prob += lpSum(x[(i,k)] for k in jobs) == 1

# Initial conditions
for j in machines:
    prob += s[0,j] == 0  # Dummy job 0 finishes at time 0

for k in jobs:
    prob += s[k,0] == 0  # Dummy machine 0 has zero processing time

# Machine timing constraints (take into acount the maximum betwen the previous machine of the same job and the next machine of the previous job)
for j in range(1, M+1):
    for k in jobs:
        prob += s[k,j] >= s[k,j-1] + lpSum(x[(i,k)] * p[(i,j)] for i in jobs)
        prob += s[k,j] >= s[k-1,j] + lpSum(x[(i,k)] * p[(i,j)] for i in jobs)

# Objective: Minimize makespan
prob += s[N,M]

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=True))

# Store solution in a dictionary sch_dict = {(job, machine): start_time} and order_dict = {order: job} = {1: jobname, 2: jobname, ...}
sch_dict = {}
order_dict = {}
x_names_ranges = {v: k for k, v in x.items()}
s_names_ranges = {v: k for k, v in s.items()}

for v in prob.variables():
    if v.varValue > 0:
        value = v.varValue
        if v.name[0] == 'x':
            (i, k) = x_names_ranges[v]
            order_dict[k] = i
        elif v.name[0] == 's':
            (k, j) = s_names_ranges[v]
            sch_dict[(k, j)] = value
order_dict = dict(sorted(order_dict.items()))

# Decode the results to match original names
# result_order: {order: job} = {1: jobname, 2: jobname, ...} as in job_names (['TTC', 'TTL', 'C', 'P', 'MM'])
result_order = {k: job_names[v-1] for k, v in order_dict.items()}

# results_endtime: {(k, machinename) : end_time_value } as in machine_names (['A1', 'A2', 'A3'])
results_endtime = {(k, machine_names[j-1]): v for (k, j), v in sch_dict.items()}

# New dictionary (order, machinename, jobname) : end_time_value, sorted by order and then by machine
# To do that, start from results_endtime and modify the keys (order, machinename) to add the jobname from result_order[(order, jobname)]
results_detailed = {}
for (k, j), v in sch_dict.items():
    results_detailed[(k, machine_names[j-1], result_order[k])] = v

# Print objective function value
print('####################################')
print("Objective value: ", prob.objective.value())
print('#----------------------------------#')
# Print the results
print('Raw solutions:')
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print('#----------------------------------#')
print('Processed solutions:')
print('Order of jobs:', order_dict)
print('End times:', sch_dict)
print('#----------------------------------#')
print('Decoded solutions:')
print('Order of jobs:', result_order)
print('End times:', results_endtime)
print('Detailed schedule:', results_detailed)
print('#----------------------------------#')
print(f"Solution Status: {LpStatus[prob.status]}")



####################################
Objective value:  35.0
#----------------------------------#
Raw solutions:
s_(1,_1) = 2.0
s_(1,_2) = 10.0
s_(1,_3) = 13.0
s_(2,_1) = 6.0
s_(2,_2) = 12.0
s_(2,_3) = 20.0
s_(3,_1) = 12.0
s_(3,_2) = 22.0
s_(3,_3) = 26.0
s_(4,_1) = 18.0
s_(4,_2) = 26.0
s_(4,_3) = 31.0
s_(5,_1) = 23.0
s_(5,_2) = 29.0
s_(5,_3) = 35.0
x_(1,_3) = 1.0
x_(2,_1) = 1.0
x_(3,_4) = 1.0
x_(4,_2) = 1.0
x_(5,_5) = 1.0
#----------------------------------#
Processed solutions:
Order of jobs: {1: 2, 2: 4, 3: 1, 4: 3, 5: 5}
End times: {(1, 1): 2.0, (1, 2): 10.0, (1, 3): 13.0, (2, 1): 6.0, (2, 2): 12.0, (2, 3): 20.0, (3, 1): 12.0, (3, 2): 22.0, (3, 3): 26.0, (4, 1): 18.0, (4, 2): 26.0, (4, 3): 31.0, (5, 1): 23.0, (5, 2): 29.0, (5, 3): 35.0}
#----------------------------------#
Decoded solutions:
Order of jobs: {1: 'TTL', 2: 'P', 3: 'TTC', 4: 'C', 5: 'MM'}
End times: {(1, 'A1'): 2.0, (1, 'A2'): 10.0, (1, 'A3'): 13.0, (2, 'A1'): 6.0, (2, 'A2'): 12.0, (2, 'A3'): 20.0, (3, 'A1'): 12.0, (3, '

In [5]:
from glpk import glpk, GLPK

solver_list = listSolvers(onlyAvailable=True)
print(solver_list)


['GLPK_CMD', 'CPLEX_CMD', 'CPLEX_PY', 'PULP_CBC_CMD']


In [2]:

# Try different solvers glpk pyscipopt highspy
try:
    # Try GLPK first
    print("Attempting to solve with GLPK...")
    solver = GLPK_CMD(msg=True, options=['--pcost'])
    prob.solve(solver)
except:
    try:
        # Try SCIP if GLPK fails
        print("\nAttempting to solve with SCIP...")
        solver = SCIP_CMD(msg=True)
        prob.solve(solver)
    except:
        try:
            # Try HiGHS if SCIP fails
            print("\nAttempting to solve with HiGHS...")
            solver = HiGHS_CMD(msg=True)
            prob.solve(solver)
        except:
            # Fall back to CBC
            print("\nFalling back to CBC...")
            solver = PULP_CBC_CMD(msg=True)
            prob.solve(solver)

Attempting to solve with GLPK...


In [3]:
from glpk import glpk, GLPK

solver_list = listSolvers(onlyAvailable=True)
print(solver_list)

solver = GLPK_CMD(msg=True, options=['--pcost', '--simplex'])
prob.solve(solver)
# ['GLPK_CMD', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD', 'COIN_CMD']

print('\n####################################\n')
print("Objective value: ", prob.objective.value())
print('#----------------------------------#')
print('Order of jobs:', result_order)
print('#----------------------------------#')
print('End times:', results_endtime)
print('#----------------------------------#')
print('Detailed schedule:', results_detailed)
print('#----------------------------------#')
print(f"Solution Status: {LpStatus[prob.status]}")
print('\n####################################\n')

NameError: name 'listSolvers' is not defined

In [ ]:
# PRINT
# SAVE SAME PRINTED INFO TO FILE report_file_name
with open(report_file_name, 'w') as f:
    f.write('\n#################################\n')
    f.write('\n###-------- Results --------###\n')
    f.write('\n#################################\n')
    print('\n#----------------------------------#\n')
    f.write('Solver: '+report['solver']+'\n')
    print('\n#----------------------------------#\n')
    f.write('\nLog:\n')
    print('\n#----------------------------------#\n')
    f.write(log)
    print('\n#----------------------------------#\n')
    f.write('\nResults:\n')
    print('\n#----------------------------------#\n')
    f.write(df_order.to_string())
    f.write(df_detailed.to_string())
    print('\n#----------------------------------#\n')
    f.write('\nReport:\n')
    print('\n#----------------------------------#\n')
    for key, values in report.items():
        f.write(key+': '+str(values)+'\n')
    print('\n#----------------------------------#\n')
    print('\n#----------------------------------#\n')
    f.close()

In [1]:
glpsol --help

NameError: name 'glpsol' is not defined